## ติดตั้งและนำเข้าไลบรารีที่จำเป็น

In [ ]:
!pip install ultralytics gradio

In [ ]:
from ultralytics import YOLO
import torch
import cv2
import gradio as gr
import json
import numpy as np


In [ ]:
import glob

train_imgs = glob.glob("/path/to/dataset/images/train/*.jpg")
valid_imgs = glob.glob("/path/to/dataset/images/val/*.jpg")
#test_imgs = glob.glob("football-players-detection-9/test/images/*.jpg")

print("Training images:", len(train_imgs))
print("validation images:", len(valid_imgs))
#print("test images:", len(test_imgs))

In [ ]:
model = YOLO("yolov8m.yaml").load("yolov8m.pt")

In [ ]:
model

In [ ]:
# Train the model
model.train(data="/path/to/data.yaml", epochs=100, imgsz=640)


## โหลดโมเดลที่ฝึกสอนแล้วสำหรับการทำนาย

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = YOLO("/path/to/best.pt").to(device)  # แทนที่ด้วย path ที่แท้จริง
model.eval()

In [ ]:
# def capture_frame():
#     # เปิดการเชื่อมต่อกับ webcam
#     cap = cv2.VideoCapture(0)
#     ret, frame = cap.read()
#     if not ret:
#         return None
#     # ปิดการเชื่อมต่อกับ webcam
#     cap.release()
#     # เปลี่ยนสีจาก BGR เป็น RGB
#     frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#     return frame

In [ ]:

def detect_ingredients(frame):
    # frame = capture_frame()
    # cap = cv2.VideoCapture(1)
    # ret, frame = cap.read()
    # cap.release()

    if frame is None:
        return None, json.dumps({"name": [], "state": "Haven't"})

    # แปลงภาพจาก BGR เป็น RGB
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # ปรับขนาดของภาพให้เหมาะสม (640x640)
    # img_resized = cv2.resize(img, (640, 640))
    img_resized = img

    results = model(img_resized)

    ingredients = []
    for result in results:
        for det in result.boxes:
            cls = int(det.cls)
            label = model.names[cls]
            ingredients.append(label)

            # วาดกรอบรอบวัตถุที่ตรวจจับได้
            xyxy = det.xyxy[0].cpu().numpy().astype(int)
            cv2.rectangle(frame, (xyxy[0], xyxy[1]), (xyxy[2], xyxy[3]), (0, 255, 0), 2)
            cv2.putText(frame, label, (xyxy[0], xyxy[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    state = "Have" if len(ingredients) > 0 else "Haven't"
    json_result = json.dumps({"name": ingredients, "state": state})

    return  json_result




In [ ]:
# def gradio_interface(image):
#     json_result = detect_ingredients(image)
#     return json_result
def gradio_interface(frame):
    json_result = detect_ingredients(frame)
    return  json_result

In [ ]:
# interface = gr.Interface(fn=gradio_interface,
#                          inputs=gr.Image(type="numpy", label="ใส่รูปภาพของคุณ"),
#                          outputs=[gr.Textbox(label="ผลลัพธ์ JSON")])

# interface.launch()

interface = gr.Interface(fn=gradio_interface,
                         inputs= gr.Image( type="numpy", label="ถ่ายรูปจาก webcam"), #sources=["webcam"],
                         outputs=[
                                  gr.Textbox(label="ผลลัพธ์ JSON")])

interface.launch()


